<a href="https://colab.research.google.com/github/jhuarancca/ASU_DataMining/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

dfIns = pd.read_csv ('InsulinData.csv')
dfCGM = pd.read_csv ('CGMData.csv')
dfInsP2 = pd.read_csv ('Insulin_patient2.csv')
dfCGMP2 = pd.read_csv ('CGM_patient2.csv')

dfIns['Datetime']=pd.to_datetime(dfIns['Date'] + ' ' + dfIns['Time'])
dfCGM['Datetime']=pd.to_datetime(dfCGM['Date'] + ' ' + dfCGM['Time'])
dfIns.rename(columns={"BWZ Carb Input (grams)": "Carb"},inplace=True)

dfInsP2['Datetime']=pd.to_datetime(dfInsP2['Date'] + ' ' + dfInsP2['Time'])
dfCGMP2['Datetime']=pd.to_datetime(dfCGM['Date'] + ' ' + dfCGMP2['Time'])
dfInsP2.rename(columns={"BWZ Carb Input (grams)": "Carb"},inplace=True)

dfMeals=dfIns
dfMeals=dfMeals.drop(dfMeals[dfMeals['Carb'].isna()].index)
dfMeals = dfMeals.drop(dfMeals[(dfMeals['Carb']==0.0)].index)
dfMeals=dfMeals[["Datetime", "Carb"]]
dfMeals=dfMeals.reset_index(drop=True)
dfMeals = dfMeals.sort_values(by=['Datetime'], ascending=True)
dfMeals['Delete']='N'

dfMealsP2=dfInsP2
dfMealsP2=dfMealsP2.drop(dfMealsP2[dfMealsP2['Carb'].isna()].index)
dfMealsP2 = dfMealsP2.drop(dfMealsP2[(dfMealsP2['Carb']==0.0)].index)
dfMealsP2=dfMealsP2[["Datetime", "Carb"]]
dfMealsP2=dfMealsP2.reset_index(drop=True)
dfMealsP2 = dfMealsP2.sort_values(by=['Datetime'], ascending=True)
dfMealsP2['Delete']='N'

from datetime import datetime, timedelta

dfMealsTmp=dfMeals
dfMealsTmp = dfMealsTmp.sort_values(by=['Datetime'], ascending=True)
dfMealsTmp=dfMealsTmp.reset_index(drop=True)

for index, row in dfMealsTmp.iterrows():
   ld_DateTime1=dfMealsTmp.at[index, 'Datetime']
   
   if index<(len(dfMealsTmp.index)-1):
    #print(index,len(dfMeals.index))
    ld_DateTime2=dfMealsTmp.at[index+1,'Datetime']
    ld_DateTime3=ld_DateTime1  + timedelta(hours=2)
    if ld_DateTime2<=ld_DateTime3:
      dfMeals.at[index,'Delete']='Y'
      #print(index,row['Datetime'], row['Carb'],ld_DateTime2,ld_DateTime3)
dfMeals01=dfMeals[(dfMeals['Delete']=='N')]
#dfMeals=dfMeals.drop(dfMeals[(dfMeals['Delete']=='Y')].index)
dfMeals01=dfMeals01.reset_index(drop=True)


dfMealsTmpP2=dfMealsP2
dfMealsTmpP2=dfMealsTmpP2.sort_values(by=['Datetime'], ascending=True)
dfMealsTmpP2=dfMealsTmpP2.reset_index(drop=True)

for index, row in dfMealsTmpP2.iterrows():
   ld_DateTime1=dfMealsTmpP2.at[index, 'Datetime']
   
   if index<(len(dfMealsTmpP2.index)-1):
    #print(index,len(dfMeals.index))
    ld_DateTime2=dfMealsTmpP2.at[index+1,'Datetime']
    ld_DateTime3=ld_DateTime1  + timedelta(hours=2)
    if ld_DateTime2<=ld_DateTime3:
      dfMealsP2.at[index,'Delete']='Y'
      #print(index,row['Datetime'], row['Carb'],ld_DateTime2,ld_DateTime3)
dfMeals01P2=dfMeals[(dfMeals['Delete']=='N')]
#dfMeals=dfMeals.drop(dfMeals[(dfMeals['Delete']=='Y')].index)
dfMeals01P2=dfMeals01.reset_index(drop=True)

#dfMeals01P2

dfCGM.rename(columns={"Sensor Glucose (mg/dL)": "Glucose"},inplace=True)
dfCGM = dfCGM.drop(dfCGM[dfCGM['Glucose'].isna()].index)
dfCGM=dfCGM[["Datetime", "Glucose"]]
dfCGM=dfCGM.reset_index(drop=True)
dfCGM = dfCGM.sort_values(by=['Datetime'], ascending=True)
dfCGM=dfCGM.reset_index(drop=True)

dfCGMP2.rename(columns={"Sensor Glucose (mg/dL)": "Glucose"},inplace=True)
dfCGMP2 = dfCGMP2.drop(dfCGMP2[dfCGMP2['Glucose'].isna()].index)
dfCGMP2=dfCGMP2[["Datetime", "Glucose"]]
dfCGMP2=dfCGMP2.reset_index(drop=True)
dfCGMP2=dfCGMP2.sort_values(by=['Datetime'], ascending=True)
dfCGMP2=dfCGMP2.reset_index(drop=True)

dfMealData = pd.DataFrame()
li_row=0
for index, row in dfMeals01.iterrows():
   ld_DateTime=dfMeals01.at[index, 'Datetime']
   ld_DateTimeFrom=dfMeals01.at[index, 'Datetime']+ timedelta(hours=-0.5)
   ld_DateTimeTo=dfMeals01.at[index, 'Datetime']+ timedelta(hours=2)
   #dfCGM
   #print(index,ld_DateTime,ld_DateTimeFrom,ld_DateTimeTo)
   dfCGM01=dfCGM.loc[(dfCGM['Datetime'] >= ld_DateTimeFrom) & (dfCGM['Datetime'] <= ld_DateTimeTo)]
   #if li_row==10:
   # print(dfCGM01)
   #print(len(dfCGM01.index) )
   if len(dfCGM01.index)==30:
    li_column=1
    for ind in dfCGM01.index:
      #print(dfCGM01['Glucose'][ind])
      #print(ind,dfCGM01.index)
      dfMealData.at[li_row,li_column]=dfCGM01['Glucose'][ind]
      li_column=li_column+1
    #dfCGM01=pd.pivot(data=dfCGM01,index='Id',columns='Id',values='Glucose')
    li_row=li_row+1
dfMealData.columns = ['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21','22', '23', '24','25', '26', '27','28', '29', '30']   

dfMealDataP2 = pd.DataFrame()
li_row=0
for index, row in dfMeals01P2.iterrows():
   ld_DateTime=dfMeals01P2.at[index, 'Datetime']
   ld_DateTimeFrom=dfMeals01P2.at[index, 'Datetime']+ timedelta(hours=-0.5)
   ld_DateTimeTo=dfMeals01P2.at[index, 'Datetime']+ timedelta(hours=2)
   #dfCGM
   #print(index,ld_DateTime,ld_DateTimeFrom,ld_DateTimeTo)
   dfCGM01P2=dfCGMP2.loc[(dfCGMP2['Datetime'] >= ld_DateTimeFrom) & (dfCGMP2['Datetime'] <= ld_DateTimeTo)]
   #if li_row==10:
   # print(dfCGM01)
   #print(len(dfCGM01.index) )
   if len(dfCGM01P2.index)==30:
    li_column=1
    for ind in dfCGM01P2.index:
      #print(dfCGM01['Glucose'][ind])
      #print(ind,dfCGM01.index)
      dfMealDataP2.at[li_row,li_column]=dfCGM01P2['Glucose'][ind]
      li_column=li_column+1
    #dfCGM01=pd.pivot(data=dfCGM01,index='Id',columns='Id',values='Glucose')
    li_row=li_row+1
dfMealDataP2.columns = ['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21','22', '23', '24','25', '26', '27','28', '29', '30']   


dfNoMealData = pd.DataFrame()
#dfCGM01 = pd.DataFrame()
li_row=0
for index, row in dfMeals01.iterrows():
   ld_DateTime=dfMeals01.at[index, 'Datetime']
   ld_DateTimeFrom=dfMeals01.at[index, 'Datetime']+ timedelta(hours=2)
   ld_DateTimeTo=dfMeals01.at[index, 'Datetime']+ timedelta(hours=4)
   if index<(len(dfMeals01.index)-1):
    ld_DateTimeNext=dfMeals01.at[index+1,'Datetime']
   else:
    ld_DateTimeNext= max(dfMeals01['Datetime'])
      
   #print("next",ld_DateTimeNext)
   while ld_DateTimeTo<=ld_DateTimeNext:
      li_count=0
      dfMeals02=dfMeals01.loc[(dfMeals01['Datetime'] >= ld_DateTimeFrom) & (dfMeals01['Datetime'] <= ld_DateTimeTo)]
      li_count=len(dfMeals02.index)
      if li_count==0:
        #print(li_count)
        ###
        dfCGM02=dfCGM.loc[(dfCGM['Datetime'] >= ld_DateTimeFrom) & (dfCGM['Datetime'] <= ld_DateTimeTo)]
        #if li_row==0:
          #print(dfCGM02)
   
        if len(dfCGM02.index)==24:
          li_column=1
          for ind in dfCGM02.index:
            #print(dfCGM01['Glucose'][ind])
            #print(ind,dfCGM01.index)
            dfNoMealData.at[li_row,li_column]=dfCGM02['Glucose'][ind]
            li_column=li_column+1
          #dfCGM01=pd.pivot(data=dfCGM01,index='Id',columns='Id',values='Glucose')
          li_row=li_row+1
        ###
        ld_DateTimeFrom=ld_DateTimeTo
        ld_DateTimeTo=ld_DateTimeTo+ timedelta(hours=2)
      else:
        #print(li_count,"break",)
        break
      #print(ld_DateTimeFrom,ld_DateTimeTo)
dfNoMealData.columns = ['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21','22', '23', '24']   

dfNoMealDataP2 = pd.DataFrame()
li_row=0
for index, row in dfMeals01P2.iterrows():
   ld_DateTime=dfMeals01P2.at[index, 'Datetime']
   ld_DateTimeFrom=dfMeals01P2.at[index, 'Datetime']+ timedelta(hours=2)
   ld_DateTimeTo=dfMeals01P2.at[index, 'Datetime']+ timedelta(hours=4)
   if index<(len(dfMeals01P2.index)-1):
    ld_DateTimeNext=dfMeals01P2.at[index+1,'Datetime']
   else:
    ld_DateTimeNext= max(dfMeals01P2['Datetime'])
      
   #print("next",ld_DateTimeNext)
   while ld_DateTimeTo<=ld_DateTimeNext:
      li_count=0
      dfMeals02P2=dfMeals01P2.loc[(dfMeals01P2['Datetime'] >= ld_DateTimeFrom) & (dfMeals01P2['Datetime'] <= ld_DateTimeTo)]
      li_count=len(dfMeals02P2.index)
      if li_count==0:
        #print(li_count)
        ###
        dfCGM02P2=dfCGMP2.loc[(dfCGMP2['Datetime'] >= ld_DateTimeFrom) & (dfCGMP2['Datetime'] <= ld_DateTimeTo)]
        #if li_row==0:
          #print(dfCGM02P2)
   
        if len(dfCGM02P2.index)==24:
          li_column=1
          for ind in dfCGM02P2.index:
            #print(dfCGM01['Glucose'][ind])
            #print(ind,dfCGM01.index)
            dfNoMealDataP2.at[li_row,li_column]=dfCGM02P2['Glucose'][ind]
            li_column=li_column+1
          #dfCGM01=pd.pivot(data=dfCGM01,index='Id',columns='Id',values='Glucose')
          li_row=li_row+1
        ###
        ld_DateTimeFrom=ld_DateTimeTo
        ld_DateTimeTo=ld_DateTimeTo+ timedelta(hours=2)
      else:
        #print(li_count,"break",)
        break
      #print(ld_DateTimeFrom,ld_DateTimeTo)
   
dfNoMealDataP2.columns = ['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21','22', '23', '24']  

dfMealData = pd.concat([dfMealData, dfMealDataP2],ignore_index=True)
dfNoMealData = pd.concat([dfNoMealData, dfNoMealDataP2],ignore_index=True)

import numpy as np
import matplotlib.pyplot as plotter
from scipy.fftpack import fft

dfMealMatrix = pd.DataFrame()

#dfMealData.info()
ld_feat01=0.00
ld_feat02=0.00
ld_feat03=0.00
ld_feat04=0.00
ld_feat05=0.00
ld_feat06=0.00
ld_feat07=0.00
ld_feat08=0.00

for index,row in dfMealData.iterrows(): 
    ld_maxMeal=max(row)
    ld_startMeal=dfMealData.at[index, '01']
    ld_meal=dfMealData.at[index, '07']
    #ld_feat01 *****************************
    ld_feat01=0.00
    x = range(0,30)
    ld_row=0
    li_indexMax=0
    for n in x:
      ld_row=row.get(n,0)
      if ld_row>=ld_maxMeal:
        li_indexMax=n+1
    #print("IndexMax",li_indexMax)
    #print(row)
    if li_indexMax>7:
      ld_feat01=(li_indexMax-7)*5
    else:   
      #Average 25  *************************************************************
      ld_feat01=(25-7)*5
    #print("ld_feat01",ld_feat01)
    #ld_feat02 *****************************
    ld_feat02=0.00
    ld_feat02=(ld_maxMeal - ld_meal)/ld_meal

    #print("ld_feat02",ld_feat02)
    #row.loc[row['column_name'] == ld_max]
    #row.iloc[:,1]
    #print(ld_max,ld_meal)
    
    #ld_feat03 *****************************
    #ld_feat04 *****************************
    #ld_feat05 *****************************
    #ld_feat06 *****************************
    ld_feat03=0.00
    ld_feat04=0.00
    ld_feat05=0.00
    ld_feat06=0.00

    #la_signal = np.array(row, dtype=float)
    #fourier = np.fft.fft(la_signal)
    #n = la_signal.size
    #sampleRate = 30
    #timestep = 0.1/sampleRate
    #la_freq = np.fft.fftfreq(n, d=timestep)

    la_sp = np.fft.fft(row)
    la_freq = np.fft.fftfreq(row.shape[-1])
    
    #la_freq = np.fft.fft(x)

    ld_feat03=abs(la_freq[0].real)
    ld_feat04=abs(la_freq[1].real)
    ld_feat05=abs(la_freq[2].real)
    ld_feat06=abs(la_freq[3].real)
    #ld_feat07 *****************************
    #First data point until max datapoint
    #ld_feat07 = (y2-y1)/(x2-x1)
    ld_feat07 = (ld_maxMeal-ld_startMeal)/(li_indexMax-0)
    
    #ld_feat08 *****************************
    ld_feat08=row.std(axis=0)

    dfMealMatrix.at[index,"feat01"]=ld_feat01
    dfMealMatrix.at[index,"feat02"]=ld_feat02
    dfMealMatrix.at[index,"feat03"]=ld_feat03
    dfMealMatrix.at[index,"feat04"]=ld_feat04
    dfMealMatrix.at[index,"feat05"]=ld_feat05
    dfMealMatrix.at[index,"feat06"]=ld_feat06
    dfMealMatrix.at[index,"feat07"]=ld_feat07
    dfMealMatrix.at[index,"feat08"]=ld_feat08
    
#row
#dfMealMatrix

dfNoMealMatrix = pd.DataFrame()

ld_feat01=0.00
ld_feat02=0.00
ld_feat03=0.00
ld_feat04=0.00
ld_feat05=0.00
ld_feat06=0.00
ld_feat07=0.00
ld_feat08=0.00

for index,row in dfNoMealData.iterrows(): 
    ld_maxMeal=max(row)
    ld_startMeal=dfNoMealData.at[index, '01']
    ld_meal=dfNoMealData.at[index, '01']
    #ld_feat01 *****************************
    ld_feat01=0.00
    x = range(0,24)
    ld_row=0
    li_indexMax=0
    for n in x:
      ld_row=row.get(n,0)
      if ld_row>=ld_maxMeal:
        li_indexMax=n+1
    #print("IndexMax",li_indexMax)
    #print(row)
    if li_indexMax>0:
      ld_feat01=(li_indexMax)*5
    else:   
      #Average 15  *************************************************************
      ld_feat01=(15-7)*5
    #print("ld_feat01",ld_feat01)
    #ld_feat02 *****************************
    ld_feat02=0.00
    ld_feat02=(ld_maxMeal - ld_meal)/ld_meal

    #print("ld_feat02",ld_feat02)
    #row.loc[row['column_name'] == ld_max]
    #row.iloc[:,1]
    #print(ld_max,ld_meal)
    
    #ld_feat03 *****************************
    #ld_feat04 *****************************
    #ld_feat05 *****************************
    #ld_feat06 *****************************
    ld_feat03=0.00
    ld_feat04=0.00
    ld_feat05=0.00
    ld_feat06=0.00

    #la_signal = np.array(row, dtype=float)
    #fourier = np.fft.fft(la_signal)
    #n = la_signal.size
    #sampleRate = 30
    #timestep = 0.1/sampleRate
    #la_freq = np.fft.fftfreq(n, d=timestep)

    la_sp = np.fft.fft(row)
    la_freq = np.fft.fftfreq(row.shape[-1])
    
    #la_freq = np.fft.fft(x)
    
    ld_feat03=abs(la_freq[0].real)
    ld_feat04=abs(la_freq[1].real)
    ld_feat05=abs(la_freq[2].real)
    ld_feat06=abs(la_freq[3].real)
    #ld_feat07 *****************************
    #First data point until max datapoint
    #ld_feat07 = (y2-y1)/(x2-x1)
    ld_feat07 = (ld_maxMeal-ld_startMeal)/(li_indexMax-0)
    
    #ld_feat08 *****************************
    ld_feat08=row.std(axis=0)

    dfNoMealMatrix.at[index,"feat01"]=ld_feat01
    dfNoMealMatrix.at[index,"feat02"]=ld_feat02
    dfNoMealMatrix.at[index,"feat03"]=ld_feat03
    dfNoMealMatrix.at[index,"feat04"]=ld_feat04
    dfNoMealMatrix.at[index,"feat05"]=ld_feat05
    dfNoMealMatrix.at[index,"feat06"]=ld_feat06
    dfNoMealMatrix.at[index,"feat07"]=ld_feat07
    dfNoMealMatrix.at[index,"feat08"]=ld_feat08
    
#row
#dfNoMealMatrix

dfMealMatrix01=dfMealMatrix
dfMealMatrix01['meal']=1

dfNoMealMatrix01=dfNoMealMatrix
dfNoMealMatrix01['meal']=0

dfMatrix = pd.concat([dfMealMatrix01, dfNoMealMatrix01],ignore_index=True)
#dfMatrix

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

samples = dfMatrix.iloc[:, :-1]
labels = dfMatrix.iloc[:, -1]

# Train the model on the extracted features
#model = train(samples, labels)

model = 'ML_MealNoMealClassifier'
acc_scores = []

# For K-Fold validation
kfold_k=5
kf=KFold(kfold_k,shuffle=True,random_state=None)

for tr, ts in kf.split(samples, labels) : 
    tr_X, ts_X = samples.iloc[tr], samples.iloc[ts]
    tr_Y, ts_Y = labels.iloc[tr], labels.iloc[ts]

    model = SVC(kernel='rbf', gamma='scale', degree=3)
    model.fit(tr_X, tr_Y)

    pred_Y = model.predict(ts_X)
    acc_scores.append(accuracy_score(ts_Y, pred_Y))

print('Accuracy: ', (np.sum(acc_scores) / kfold_k) * 100)

# Dump the trained model into a pickle file
with open('mealNoMealModel.pkl', 'wb') as file :
   pickle.dump(model, file)
   



FileNotFoundError: ignored